# **Ejercicio en clase: Web Scraping**
**ICCD753 Recuperación de Información – Prof. Iván Carrera (2024-B, EPN-FIS)**  

**Fecha de Entrega:** Viernes 24 de enero de 2025  
**Nombre:**  
- Dilan Andrade  


---

Este trabajo realiza web scraping para extraer categorías de recetas de la página web de AllRecipes, guarda la información en un archivo CSV y luego procesa las recetas extrayendo su título, descripción, ingredientes y pasos. Utiliza el modelo Word2Vec para generar vectores promedio de las recetas y permite realizar búsquedas semánticas por similitud, comparando una consulta con las recetas y mostrando los resultados más relevantes.

---



# **Web Scraping para Extraer Categorías de Recetas**
#### **Instalación de Dependencias**

Antes de ejecutar el código, asegúrate de instalar las bibliotecas necesarias:

  *!pip install requests beautifulsoup4*
#### **Explicación:**

1. **Instalación de Paquetes**: Se instalan las bibliotecas necesarias `requests` y `beautifulsoup4` para hacer las solicitudes HTTP y analizar el contenido HTML.
   
2. **Realización de la Solicitud HTTP**: Se realiza una solicitud a la URL de la página de AllRecipes con encabezados para simular un navegador real.

3. **Análisis del HTML**: Usando `BeautifulSoup`, el HTML obtenido se analiza para extraer la información deseada.

4. **Extracción de Categorías**: Se buscan los elementos `<li>` con la clase correspondiente que contienen los enlaces a las categorías de recetas. Luego, se extraen el nombre y la URL de cada categoría.

5. **Guardar Datos en CSV**: Los datos extraídos (categorías y URLs) se escriben en un archivo CSV para almacenarlos de manera estructurada.




In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# URL de la página web
url = "https://www.allrecipes.com/recipes-a-z-6735880"

# Encabezados adicionales
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.allrecipes.com/"
}

# Realizar la solicitud a la página
response = requests.get(url, headers=headers)
response.raise_for_status()  # Verificar que la solicitud fue exitosa

# Analizar el contenido HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Buscar todos los elementos <li> con la clase específica
list_items = soup.find_all('li', class_="mntl-link-list__item")

# Lista para almacenar los datos
data = []

# Extraer la categoría y el enlace de cada elemento
for item in list_items:
    link_tag = item.find('a', class_="mntl-link-list__link")
    if link_tag and 'href' in link_tag.attrs:
        categoria = link_tag.text.strip()  # Nombre de la categoría
        url = link_tag['href']  # URL
        data.append({'categoria': categoria, 'url': url})

# Guardar los datos en un archivo CSV
csv_file = "categorias_recetas.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['categoria', 'url'])
    writer.writeheader()
    writer.writerows(data)

print(f"Datos guardados en {csv_file} exitosamente.")


Datos guardados en categorias_recetas.csv exitosamente.


# **Extracción de Recetas desde Categorías Específicas**

Leemos un archivo CSV con categorías y URLs previamente extraídas, luego realiza scraping sobre cada una de esas URLs para obtener información detallada sobre las recetas, como su ID, nombre y URL. Los datos son guardados en un archivo CSV sin duplicados.

#### **Explicación:**

1. **Lectura del Archivo CSV de Categorías**: Se lee el archivo CSV que contiene las categorías y URLs extraídas previamente.

2. **Solicitud HTTP a Cada URL de Categoría**: Para cada URL de categoría, se realiza una solicitud HTTP con los mismos encabezados para obtener el contenido de la página de recetas.

3. **Análisis de las Páginas de Recetas**: Usando `BeautifulSoup`, se extraen los enlaces de las recetas dentro de la página de cada categoría.

4. **Extracción de Información de Recetas**: Se obtienen el ID de la receta, su nombre y la URL de cada receta encontrada.

5. **Evitar Duplicados**: Se asegura que cada receta sea única verificando su ID antes de agregarla a la lista de datos.

6. **Guardar los Datos en CSV**: Finalmente, los datos de las recetas (categoría, ID, nombre y URL) se guardan en un archivo CSV sin duplicados.


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Archivo CSV de entrada (categorías y URLs)
input_csv = "categorias_recetas.csv"

# Archivo CSV de salida (datos de las recetas)
output_csv = "recetas_extraidas.csv"

# Encabezados para el archivo de salida
output_headers = ['categoria', 'id_receta', 'nombre_receta', 'url_receta']

# Encabezados HTTP para la solicitud
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
}

# Set para almacenar IDs únicos de recetas y evitar duplicados
unique_ids = set()

# Lista para almacenar los datos extraídos
recipes_data = []

# Leer el archivo de categorías y procesar cada URL
with open(input_csv, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    for row in reader:
        categoria = row['categoria']
        url = row['url']

        print(f"Procesando categoría: {categoria} - URL: {url}")

        # Realizar solicitud a la URL de la categoría
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Verificar que la solicitud fue exitosa

        # Analizar el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Buscar todas las recetas en la página
        recipes = soup.find_all('a', class_='mntl-card-list-items')

        for recipe in recipes:
            recipe_url = recipe['href']  # URL de la receta
            recipe_id = recipe.get('data-doc-id')  # ID de la receta
            recipe_name = recipe.find('span', class_='card__title-text').text.strip()  # Nombre de la receta

            # Evitar duplicados verificando el ID de la receta
            if recipe_id not in unique_ids:
                unique_ids.add(recipe_id)  # Agregar el ID al set de únicos
                recipes_data.append({
                    'categoria': categoria,
                    'id_receta': recipe_id,
                    'nombre_receta': recipe_name,
                    'url_receta': recipe_url
                })

# Guardar los datos extraídos en un nuevo archivo CSV
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=output_headers)
    writer.writeheader()
    writer.writerows(recipes_data)

print(f"Datos de recetas guardados en {output_csv} exitosamente.")


Procesando categoría: Air Fryer Recipes - URL: https://www.allrecipes.com/recipes/23070/everyday-cooking/cookware-and-equipment/air-fryer/
Procesando categoría: Allrecipes Allstar Recipes - URL: https://www.allrecipes.com/recipes/16492/everyday-cooking/special-collections/allrecipes-allstars/
Procesando categoría: Angel Food Cakes - URL: https://www.allrecipes.com/recipes/385/desserts/cakes/angel-food-cake/
Procesando categoría: Antipasti - URL: https://www.allrecipes.com/recipes/102/appetizers-and-snacks/antipasto/
Procesando categoría: Appetizers and Snacks - URL: https://www.allrecipes.com/recipes/76/appetizers-and-snacks/
Procesando categoría: Apple Pie - URL: https://www.allrecipes.com/recipes/788/desserts/pies/apple-pie/
Procesando categoría: Applesauce - URL: https://www.allrecipes.com/recipes/1333/side-dish/applesauce/
Procesando categoría: Artichoke Dips - URL: https://www.allrecipes.com/recipes/14913/appetizers-and-snacks/dips-and-spreads/artichoke-dip/
Procesando categoría: 

# **Extracción Detallada de Recetas desde Enlaces**

Este script extrae información detallada de recetas (título, descripción, ingredientes y pasos) desde URLs almacenadas en un archivo CSV y guarda los datos en un nuevo archivo CSV, procesando solo los primeros 1000 enlaces.

#### **Explicación:**

1. **Lectura del CSV de Recetas**: Se lee un archivo CSV con URLs de recetas.
   
2. **Solicitud HTTP y Análisis HTML**: Para cada URL, se realiza una solicitud HTTP y se analiza el contenido HTML para extraer título, descripción, ingredientes y pasos.

3. **Validación de Datos**: Se verifica que todos los campos estén completos antes de guardar los datos.

4. **Guardar en CSV**: Los datos extraídos se almacenan en un archivo CSV, con ingredientes y pasos en formato de lista.


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Archivo de entrada con las URLs de las recetas
input_csv = "recetas_extraidas.csv"

# Archivo de salida con los datos procesados
output_csv = "Recetas_DilanAndrade.csv"

# Encabezados del archivo de salida
output_headers = ['categoria', 'titulo_receta', 'descripcion_receta', 'ingredientes', 'pasos']

# Encabezados HTTP para la solicitud
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
}

# Lista para almacenar los datos extraídos
recipes_details = []

# Leer el archivo CSV de recetas y procesar los primeros 1000 enlaces
with open(input_csv, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for i, row in enumerate(reader):
        if i >= 1000:  # Limitar a los primeros 1000 enlaces
            break

        categoria = row['categoria']
        url_receta = row['url_receta']

        print(f"Procesando receta {i + 1}: {url_receta}")

        try:
            # Realizar la solicitud a la página de la receta
            response = requests.get(url_receta, headers=headers)
            response.raise_for_status()  # Verificar que la solicitud fue exitosa

            # Analizar el contenido HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraer título de la receta
            titulo_receta_tag = soup.find('h1', class_='article-heading text-headline-400')
            titulo_receta = titulo_receta_tag.text.strip() if titulo_receta_tag else None

            # Extraer descripción de la receta
            descripcion_receta_tag = soup.find('p', class_='article-subheading text-body-100')
            descripcion_receta = descripcion_receta_tag.text.strip() if descripcion_receta_tag else None

            # Extraer ingredientes como una lista (línea por línea)
            ingredientes_list = soup.find_all('li', class_='mm-recipes-structured-ingredients__list-item')
            ingredientes = [item.text.strip() for item in ingredientes_list] if ingredientes_list else None

            # Extraer pasos de la receta
            pasos_list = soup.find_all('li', class_='comp mntl-sc-block mntl-sc-block-startgroup mntl-sc-block-group--LI')
            pasos = [step.find('p', class_='comp mntl-sc-block mntl-sc-block-html').text.strip() for step in pasos_list] if pasos_list else None

            # Validar que todos los datos requeridos existan
            if titulo_receta and descripcion_receta and ingredientes and pasos:
                # Guardar los datos extraídos
                recipes_details.append({
                    'categoria': categoria,
                    'titulo_receta': titulo_receta,
                    'descripcion_receta': descripcion_receta,
                    'ingredientes': ingredientes,  # Guardar como lista directamente
                    'pasos': pasos  # Guardar los pasos como lista
                })
            else:
                print(f"Datos incompletos en la receta {i + 1}, no se agregará.")

        except Exception as e:
            print(f"Error procesando la receta {i + 1}: {e}")
            continue

# Guardar los detalles en un archivo CSV
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(output_headers)

    for recipe in recipes_details:
        # Escribir la receta en el archivo
        writer.writerow([
            recipe['categoria'],
            recipe['titulo_receta'],
            recipe['descripcion_receta'],
            "\n".join(recipe['ingredientes']),  # Cada ingrediente en una nueva línea
            "\n".join(recipe['pasos'])  # Cada paso en una nueva línea
        ])

print(f"Datos de las recetas guardados en {output_csv} exitosamente.")


Procesando receta 1: https://www.allrecipes.com/gallery/best-air-fryer-snack-recipes/
Datos incompletos en la receta 1, no se agregará.
Procesando receta 2: https://www.allrecipes.com/gallery/air-fryer-side-dishes-for-dinner/
Datos incompletos en la receta 2, no se agregará.
Procesando receta 3: https://www.allrecipes.com/article/how-to-convert-recipes-for-an-air-fryer/
Datos incompletos en la receta 3, no se agregará.
Procesando receta 4: https://www.allrecipes.com/article/air-fryer-tips/
Datos incompletos en la receta 4, no se agregará.
Procesando receta 5: https://www.allrecipes.com/ninja-crispi-air-fryer-amazon-2024-8732423
Datos incompletos en la receta 5, no se agregará.
Procesando receta 6: https://www.allrecipes.com/recipe/283850/easy-air-fried-chicken-breast/
Procesando receta 7: https://www.allrecipes.com/air-fryer-appetizer-recipes-8746703
Datos incompletos en la receta 7, no se agregará.
Procesando receta 8: https://www.allrecipes.com/air-fryer-lemon-garlic-parmesan-chicken

# **Preparación del corpus**

Importamos las bibliotecas necesarias y descargamos los recursos de NLTK para tokenización y eliminación de stopwords, además de cargar un modelo de palabras preentrenado.

#### **Explicación:**

1. **Importamos las Bibliotecas**: Cargamos `pandas`, `numpy`, `nltk`, y `gensim` para procesamiento de texto y modelos preentrenados.
   
2. **Descargamos los Recursos NLTK**: Bajamos los recursos para tokenización y stopwords.


In [7]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import gensim.downloader as api
from numpy.linalg import norm
from tabulate import tabulate  # Para mostrar los resultados en una tabla

# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **Funciones para Procesamiento de Texto y Búsqueda de Similitudes**

Implementamos funciones para preprocesar texto, generar vectores promedio, calcular similitud entre vectores usando el coseno, y buscar recetas relevantes basadas en una consulta.

#### **Explicación:**

1. **Preprocesamiento de Texto**:
   - Eliminamos caracteres especiales y convertimos el texto a minúsculas.
   - Tokenizamos el texto en palabras.

2. **Generación de Vectores Promedio**:
   - Obtenemos el vector promedio de las palabras presentes en el modelo de palabras preentrenadas.
   - Si no se encuentran palabras válidas, devolvemos un vector nulo.

3. **Similitud del Coseno**:
   - Calculamos la similitud entre dos vectores usando el coseno, retornando un valor entre 0 y 1.

4. **Búsqueda de Recetas Relevantes**:
   - Generamos el vector de la consulta, calculamos la similitud con los vectores de las recetas y ordenamos los resultados por similitud para devolver las mejores coincidencias.


In [2]:
# Función de preprocesamiento
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Eliminar caracteres especiales y convertir a minúsculas
    tokens = word_tokenize(text)  # Tokenizar

    return tokens

# Función para generar el vector promedio de un texto
def get_vector(text, model):
    tokens = preprocess(text)
    in_vocab = [word for word in tokens if word in model]
    print(f"Tokens en vocabulario: {len(in_vocab)} de {len(tokens)}")  # Debug
    vectors = [model[word] for word in in_vocab]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # Vector nulo si no hay palabras válidas

# Función de similitud del coseno
def cosine_similarity(vec1, vec2):
    norm1 = norm(vec1)
    norm2 = norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return np.dot(vec1, vec2) / (norm1 * norm2)

# Función para buscar recetas relevantes
def search(query, data, model, top_n=5):
    query_vector = get_vector(query, model)  # Generar vector de la consulta
    data['similarity'] = data['vector'].apply(lambda vec: cosine_similarity(query_vector, vec))
    results = data.sort_values(by='similarity', ascending=False).head(top_n)  # Ordenar por similitud
    return results[['categoria', 'titulo_receta', 'descripcion_receta', 'similarity']]


# **Carga del Modelo Word2Vec**

Cargamos el modelo preentrenado de Word2Vec ("word2vec-google-news-300") para utilizarlo en tareas de análisis semántico, como generar vectores de palabras y calcular similitudes.

In [3]:
print("Cargando el modelo Word2Vec...")
word2vec_model = api.load("word2vec-google-news-300")
print("Modelo cargado exitosamente.")


Cargando el modelo Word2Vec...
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Modelo cargado exitosamente.


# **Carga y Preprocesamiento de Datos de Recetas**

Cargamos un archivo CSV con datos de recetas y creamos una nueva columna que combina el título, descripción, ingredientes y pasos de cada receta para formar un texto completo.

#### **Explicación por Pasos:**

1. **Cargar el CSV**:
   - Usamos `pandas` para leer el archivo CSV que contiene los datos de las recetas.

2. **Crear Columna de Texto Completo**:
   - Combinamos el título, descripción, ingredientes y pasos de cada receta en una nueva columna llamada `texto_completo`.

3. **Verificar Contenido**:
   - Mostramos las primeras filas para verificar que los datos se han combinado correctamente.


In [4]:
file_path = "/content/Recetas_DilanAndrade.csv"  # Cambia por la ruta de tu archivo
print("Cargando el archivo CSV...")
data = pd.read_csv(file_path)
print("Archivo cargado exitosamente.")

# Crear una nueva columna 'texto_completo' combinando las partes relevantes
data['texto_completo'] = data.apply(lambda row: f"{row['titulo_receta']} {row['descripcion_receta']} {row['ingredientes']} {row['pasos']}", axis=1)

# Verificar el contenido
print(data[['categoria', 'titulo_receta', 'texto_completo']].head())


Cargando el archivo CSV...
Archivo cargado exitosamente.
           categoria                            titulo_receta  \
0  Air Fryer Recipes            Easy Air Fryer Chicken Breast   
1  Air Fryer Recipes  Air Fryer Lemon Garlic Parmesan Chicken   
2  Air Fryer Recipes                        Air Fryer S’Mores   
3  Air Fryer Recipes                     Air Fryer Baked Yams   
4  Air Fryer Recipes     Lemon Garlic Butter Chicken Spiedini   

                                      texto_completo  
0  Easy Air Fryer Chicken Breast This air fryer c...  
1  Air Fryer Lemon Garlic Parmesan Chicken These ...  
2  Air Fryer S’Mores This recipe for air fryer s'...  
3  Air Fryer Baked Yams These air fryer baked yam...  
4  Lemon Garlic Butter Chicken Spiedini These lem...  


# **Generación de Vectores para las Recetas**

Generamos vectores promedio para cada receta utilizando el modelo Word2Vec cargado, basándonos en el texto completo de cada receta.

#### **Explicación:**

1. **Generar Vectores**:
   - Aplicamos la función `get_vector` a la columna `texto_completo` para obtener un vector promedio para cada receta.

2. **Almacenar Vectores**:
   - Los vectores generados se almacenan en una nueva columna llamada `vector`.


In [8]:
print("Generando vectores para las recetas...")
data['vector'] = data['texto_completo'].apply(lambda text: get_vector(text, word2vec_model))
print("Vectores generados exitosamente.")

Generando vectores para las recetas...
Tokens en vocabulario: 153 de 178
Tokens en vocabulario: 168 de 190
Tokens en vocabulario: 141 de 166
Tokens en vocabulario: 70 de 84
Tokens en vocabulario: 254 de 297
Tokens en vocabulario: 152 de 179
Tokens en vocabulario: 277 de 334
Tokens en vocabulario: 126 de 147
Tokens en vocabulario: 152 de 179
Tokens en vocabulario: 118 de 134
Tokens en vocabulario: 122 de 146
Tokens en vocabulario: 242 de 278
Tokens en vocabulario: 152 de 166
Tokens en vocabulario: 141 de 168
Tokens en vocabulario: 107 de 130
Tokens en vocabulario: 173 de 208
Tokens en vocabulario: 178 de 207
Tokens en vocabulario: 157 de 183
Tokens en vocabulario: 154 de 173
Tokens en vocabulario: 155 de 185
Tokens en vocabulario: 230 de 273
Tokens en vocabulario: 179 de 211
Tokens en vocabulario: 245 de 277
Tokens en vocabulario: 186 de 218
Tokens en vocabulario: 211 de 246
Tokens en vocabulario: 157 de 176
Tokens en vocabulario: 338 de 385
Tokens en vocabulario: 218 de 247
Tokens en v

# **Búsqueda de Recetas por Consulta**

Buscamos recetas relacionadas con una consulta (en este caso, "chicken") y mostramos los resultados ordenados por similitud con el término de búsqueda.

#### **Explicación:**

1. **Realizar Búsqueda**:
   - Usamos la función `search` para obtener las recetas más relevantes para la consulta dada, ordenadas por similitud.

2. **Mostrar Resultados**:
   - Convertimos los resultados en una tabla y los mostramos utilizando `tabulate` con formato bonito.
   - También mostramos la similitud promedio entre las recetas y la consulta.


In [9]:
query = "chicken"  # Cambia por tu consulta
print(f"Buscando recetas para la consulta: '{query}'...")

# Buscar todas las recetas ordenadas por similitud
results = search(query, data, word2vec_model, top_n=len(data))  # Usar todas las filas disponibles

# Mostrar los resultados en formato tabular
print("\nResultados detallados (todas las recetas):")
headers = ["Categoría", "Título de la Receta", "Descripción", "Similitud"]
table = results.values.tolist()  # Convertir los resultados a una lista de listas
print(tabulate(table, headers=headers, tablefmt="fancy_grid", numalign="center"))  # Tabular con formato

# Mostrar la similitud promedio
print("\nSimilitud promedio:", round(results['similarity'].mean(), 4))


Buscando recetas para la consulta: 'chicken'...
Tokens en vocabulario: 1 de 1

Resultados detallados (todas las recetas):
╒════════════════════════════╤══════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════╕
│ Categoría                  │ Título de la Receta    